# 📏 Measures of Dispersion
## Understanding the SPREAD of Your Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-4-biometry/notebooks/03_dispersion_measures.ipynb)

---

> *"Knowing the CENTER is not enough. How SPREAD OUT is your data?"*

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Understand **Range, Quartiles, Mean Deviation, Standard Deviation, Variance**
2. Visualize **spread** interactively
3. Know **when to use each** measure
4. Apply to **real ecological scenarios**
5. Understand the **68-95-99.7 rule** for Normal distributions

---

## 🩺 The Stethoscope Analogy

Remember our 7-step journey:
- **Step 6**: MEASURE the pattern
- Last notebook: We found the CENTER (Mean, Median, Mode)
- This notebook: We measure the SPREAD (Dispersion)

### Medical Analogy:

Just knowing average blood pressure (120/80) isn't enough. A doctor also needs to know:
- **How variable** is it? (Range: 110-130 or 90-150?)
- **How consistent** are the readings? (Low or high variability?)

Similarly for ecological data:
- Average plant height = 50cm is incomplete
- **How varied** are the plants? (All 45-55cm, or 20-80cm?)

In [ ]:
# Setup
!pip install numpy scipy plotly pandas -q

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import pandas as pd

np.random.seed(42)

print("✅ Ready to explore dispersion!")
print("📏 Let's measure the SPREAD of data!")

---

## 📊 Part 1: The Five Measures

| Measure | Formula | What It Tells You |
|---------|---------|-------------------|
| **Range** | Max - Min | Total spread (crude) |
| **Quartile Deviation** | (Q₃ - Q₁) / 2 | Spread of middle 50% |
| **Mean Deviation** | Σ\|x - μ\| / n | Average distance from mean |
| **Variance (σ²)** | Σ(x - μ)² / n | Average squared deviation |
| **Standard Deviation (σ)** | √Variance | Average deviation (same units as data) |

---

## 🎮 Part 2: Visual Comparison - Same Mean, Different Spread!

In [ ]:
# Create three datasets with SAME MEAN but DIFFERENT SPREAD
np.random.seed(42)

# All have mean = 50
low_spread = np.random.normal(50, 3, 1000)    # σ = 3 (narrow)
medium_spread = np.random.normal(50, 10, 1000) # σ = 10 (medium)
high_spread = np.random.normal(50, 20, 1000)   # σ = 20 (wide)

# Create interactive comparison
fig = go.Figure()

# Add all three distributions
datasets = [
    ("Low Spread (σ=3)", low_spread, 'green'),
    ("Medium Spread (σ=10)", medium_spread, 'blue'),
    ("High Spread (σ=20)", high_spread, 'red')
]

for i, (name, data, color) in enumerate(datasets):
    fig.add_trace(go.Histogram(
        x=data,
        nbinsx=40,
        name=name,
        marker_color=color,
        opacity=0.6,
        visible=(i==1)  # Show medium initially
    ))

# Create buttons
buttons = []
for i, (name, data, color) in enumerate(datasets):
    visible = [False, False, False]
    visible[i] = True
    
    std = np.std(data)
    range_val = np.max(data) - np.min(data)
    
    buttons.append(
        dict(
            label=name,
            method="update",
            args=[
                {"visible": visible},
                {"title": f"📏 {name}<br>Mean=50 | SD={std:.1f} | Range={range_val:.1f}"}
            ]
        )
    )

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="down",
            x=0.7,
            y=1.15,
            buttons=buttons
        )
    ],
    title="📏 Medium Spread (σ=10)<br>Mean=50 | SD=10.0 | Range=87.5",
    xaxis_title="Plant Height (cm)",
    yaxis_title="Frequency",
    height=500,
    template='plotly_white',
    xaxis_range=[0, 120]
)

fig.show()

print("\n💡 Click buttons above to compare!")
print("📍 Notice: ALL have mean = 50, but VERY different spreads!")
print("🎯 This is why we need dispersion measures!")

---

## 📐 Part 3: Range - The Simplest Measure

In [ ]:
# Example: Leaf lengths in two plant species
species_A = np.array([8, 9, 10, 11, 12, 13, 14])  # Consistent size
species_B = np.array([5, 7, 10, 13, 15, 18, 20])  # Variable size

# Calculate ranges
range_A = np.max(species_A) - np.min(species_A)
range_B = np.max(species_B) - np.min(species_B)

mean_A = np.mean(species_A)
mean_B = np.mean(species_B)

print("🌿 Species A leaf lengths (cm):", species_A)
print(f"   Mean: {mean_A:.1f} cm")
print(f"   Range: {np.min(species_A)} to {np.max(species_A)} = {range_A} cm")
print(f"   → Low variation (consistent leaf size)\n")

print("🌿 Species B leaf lengths (cm):", species_B)
print(f"   Mean: {mean_B:.1f} cm (almost same as Species A!)")
print(f"   Range: {np.min(species_B)} to {np.max(species_B)} = {range_B} cm")
print(f"   → High variation (inconsistent leaf size)\n")

# Visualize
fig = go.Figure()

fig.add_trace(go.Box(
    y=species_A,
    name='Species A',
    marker_color='lightgreen',
    boxmean='sd'
))

fig.add_trace(go.Box(
    y=species_B,
    name='Species B',
    marker_color='lightcoral',
    boxmean='sd'
))

fig.update_layout(
    title="📏 Box Plots Showing Range and Spread",
    yaxis_title="Leaf Length (cm)",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n⚠️ Problem with Range:")
print("   • Only uses 2 values (min and max)")
print("   • Very sensitive to outliers")
print("   • Ignores all data in between")

---

## 📊 Part 4: Quartiles and IQR (Interquartile Range)

In [ ]:
# Generate sample data
np.random.seed(42)
tree_heights = np.random.normal(15, 4, 100)  # 100 trees, mean=15m, sd=4m

# Calculate quartiles
Q1 = np.percentile(tree_heights, 25)
Q2 = np.percentile(tree_heights, 50)  # This is the median!
Q3 = np.percentile(tree_heights, 75)
IQR = Q3 - Q1
quartile_deviation = IQR / 2

print("🌲 Tree Heights Distribution:\n")
print(f"   Q1 (25th percentile): {Q1:.2f} m → 25% of trees are shorter")
print(f"   Q2 (50th percentile/Median): {Q2:.2f} m → Middle value")
print(f"   Q3 (75th percentile): {Q3:.2f} m → 75% of trees are shorter")
print(f"\n   IQR (Q3 - Q1): {IQR:.2f} m → Range of middle 50%")
print(f"   Quartile Deviation: {quartile_deviation:.2f} m\n")

# Create box plot with annotations
fig = go.Figure()

fig.add_trace(go.Box(
    y=tree_heights,
    name='Tree Heights',
    marker_color='lightblue',
    boxmean='sd',
    boxpoints='outliers'
))

# Add annotations
fig.add_annotation(x=0.5, y=Q1, text=f"Q1 = {Q1:.1f}m", showarrow=True, arrowhead=2)
fig.add_annotation(x=0.5, y=Q2, text=f"Q2 (Median) = {Q2:.1f}m", showarrow=True, arrowhead=2)
fig.add_annotation(x=0.5, y=Q3, text=f"Q3 = {Q3:.1f}m", showarrow=True, arrowhead=2)

# Add IQR bracket
fig.add_shape(
    type="line",
    x0=0.8, x1=0.8,
    y0=Q1, y1=Q3,
    line=dict(color="red", width=3)
)
fig.add_annotation(
    x=0.9, y=(Q1+Q3)/2,
    text=f"IQR = {IQR:.1f}m<br>(middle 50%)",
    showarrow=False,
    font=dict(color="red", size=12)
)

fig.update_layout(
    title="📊 Box Plot: Understanding Quartiles and IQR",
    yaxis_title="Tree Height (m)",
    height=600,
    template='plotly_white',
    showlegend=False
)

fig.show()

print("💡 Box Plot Components:")
print("   • Box = IQR (middle 50% of data)")
print("   • Line in box = Median (Q2)")
print("   • Whiskers = Extend to min/max (or 1.5×IQR)")
print("   • Dots = Outliers beyond whiskers")

---

## 📏 Part 5: Mean Deviation

In [ ]:
# Small example for clarity
egg_counts = np.array([8, 10, 12, 14, 16])  # Number of eggs per nest

mean_eggs = np.mean(egg_counts)
deviations = egg_counts - mean_eggs
abs_deviations = np.abs(deviations)
mean_deviation = np.mean(abs_deviations)

print("🥚 Eggs per Nest: ", egg_counts)
print(f"\n📍 Mean: {mean_eggs:.1f} eggs\n")

print("📊 Calculating Mean Deviation (Step-by-step):\n")
print("Nest | Eggs | Deviation | Absolute Deviation")
print("-----|------|-----------|-------------------")
for i, eggs in enumerate(egg_counts):
    print(f"  {i+1}  |  {eggs}   |   {deviations[i]:+.1f}     |      {abs_deviations[i]:.1f}")

print(f"\nSum of absolute deviations: {np.sum(abs_deviations):.1f}")
print(f"Number of nests: {len(egg_counts)}")
print(f"\nMean Deviation = {np.sum(abs_deviations):.1f} / {len(egg_counts)} = {mean_deviation:.2f} eggs")

print(f"\n💡 Interpretation:")
print(f"   On average, nests deviate {mean_deviation:.2f} eggs from the mean ({mean_eggs:.1f})")

# Visualize deviations
fig = go.Figure()

# Bar chart
fig.add_trace(go.Bar(
    x=[f"Nest {i+1}" for i in range(len(egg_counts))],
    y=egg_counts,
    marker_color='lightblue',
    name='Egg Count'
))

# Add mean line
fig.add_hline(y=mean_eggs, line_dash="dash", line_color="red",
              annotation_text=f"Mean = {mean_eggs:.1f}")

fig.update_layout(
    title="🥚 Egg Counts and Deviations from Mean",
    xaxis_title="Nest",
    yaxis_title="Number of Eggs",
    height=400,
    template='plotly_white',
    showlegend=False
)

fig.show()

---

## 🎯 Part 6: Variance and Standard Deviation - The Gold Standards

In [ ]:
# Using the same egg data
print("🥚 Eggs per Nest: ", egg_counts)
print(f"Mean: {mean_eggs:.1f} eggs\n")

# Calculate variance and SD
squared_deviations = (egg_counts - mean_eggs) ** 2
variance = np.mean(squared_deviations)
std_dev = np.sqrt(variance)

print("📊 Calculating Variance & Standard Deviation:\n")
print("Nest | Eggs | Deviation | Squared Deviation")
print("-----|------|-----------|------------------")
for i, eggs in enumerate(egg_counts):
    print(f"  {i+1}  |  {eggs}   |   {deviations[i]:+.1f}     |      {squared_deviations[i]:.1f}")

print(f"\nSum of squared deviations: {np.sum(squared_deviations):.1f}")
print(f"Number of nests: {len(egg_counts)}")
print(f"\n✅ Variance (σ²) = {np.sum(squared_deviations):.1f} / {len(egg_counts)} = {variance:.2f} eggs²")
print(f"✅ Standard Deviation (σ) = √{variance:.2f} = {std_dev:.2f} eggs")

print(f"\n💡 Why square the deviations?")
print("   • Positive and negative deviations don't cancel out")
print("   • Larger deviations get MORE weight (squared)")
print("   • Standard deviation returns to original units")

print(f"\n🎯 Interpretation:")
print(f"   Egg counts typically vary by ±{std_dev:.2f} eggs from the mean")

---

## 📈 Part 7: The 68-95-99.7 Rule (Empirical Rule)

In [ ]:
# For NORMAL distributions only!
np.random.seed(42)
butterfly_wingspan = np.random.normal(50, 8, 10000)  # mean=50mm, sd=8mm

mean = 50
sd = 8

# Calculate percentages
within_1sd = np.sum((butterfly_wingspan >= mean-sd) & (butterfly_wingspan <= mean+sd)) / len(butterfly_wingspan) * 100
within_2sd = np.sum((butterfly_wingspan >= mean-2*sd) & (butterfly_wingspan <= mean+2*sd)) / len(butterfly_wingspan) * 100
within_3sd = np.sum((butterfly_wingspan >= mean-3*sd) & (butterfly_wingspan <= mean+3*sd)) / len(butterfly_wingspan) * 100

# Create visualization
x = np.linspace(mean - 4*sd, mean + 4*sd, 1000)
y = stats.norm.pdf(x, mean, sd)

fig = go.Figure()

# Base curve
fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='lines',
    line=dict(color='black', width=2),
    name='Distribution'
))

# Fill areas
# ±1σ (68%)
x_1sd = x[(x >= mean-sd) & (x <= mean+sd)]
y_1sd = stats.norm.pdf(x_1sd, mean, sd)
fig.add_trace(go.Scatter(
    x=x_1sd, y=y_1sd,
    fill='tozeroy',
    fillcolor='rgba(0, 255, 0, 0.3)',
    line=dict(width=0),
    name='±1σ (68%)'
))

# ±2σ (95%)
x_2sd = x[(x >= mean-2*sd) & (x <= mean+2*sd)]
y_2sd = stats.norm.pdf(x_2sd, mean, sd)
fig.add_trace(go.Scatter(
    x=x_2sd, y=y_2sd,
    fill='tozeroy',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(width=0),
    name='±2σ (95%)'
))

# ±3σ (99.7%)
x_3sd = x[(x >= mean-3*sd) & (x <= mean+3*sd)]
y_3sd = stats.norm.pdf(x_3sd, mean, sd)
fig.add_trace(go.Scatter(
    x=x_3sd, y=y_3sd,
    fill='tozeroy',
    fillcolor='rgba(255, 0, 0, 0.1)',
    line=dict(width=0),
    name='±3σ (99.7%)'
))

# Add vertical lines
fig.add_vline(x=mean, line_dash="dash", line_color="black", annotation_text="μ")
fig.add_vline(x=mean-sd, line_dash="dot", line_color="green", annotation_text="μ-σ")
fig.add_vline(x=mean+sd, line_dash="dot", line_color="green", annotation_text="μ+σ")
fig.add_vline(x=mean-2*sd, line_dash="dot", line_color="blue", annotation_text="μ-2σ")
fig.add_vline(x=mean+2*sd, line_dash="dot", line_color="blue", annotation_text="μ+2σ")

fig.update_layout(
    title="🦋 The 68-95-99.7 Rule (Empirical Rule)<br><sub>Butterfly Wingspan: μ=50mm, σ=8mm</sub>",
    xaxis_title="Wingspan (mm)",
    yaxis_title="Probability Density",
    height=600,
    template='plotly_white'
)

fig.show()

print("\n📊 The 68-95-99.7 Rule (for Normal distributions):\n")
print(f"   • {within_1sd:.1f}% of butterflies have wingspan between {mean-sd}mm and {mean+sd}mm (μ ± 1σ)")
print(f"   • {within_2sd:.1f}% are between {mean-2*sd}mm and {mean+2*sd}mm (μ ± 2σ)")
print(f"   • {within_3sd:.1f}% are between {mean-3*sd}mm and {mean+3*sd}mm (μ ± 3σ)")

print(f"\n💡 Practical Use:")
print(f"   If you catch a butterfly with wingspan 66mm (μ + 2σ):")
print(f"   → It's in the top 2.5% (unusual but not impossible)")
print(f"   → Values beyond μ ± 3σ are VERY rare (<0.3%)")

---

## 🌍 Part 8: Real Ecological Application

In [ ]:
# Compare two populations with different dispersions
np.random.seed(42)

# Population A: Controlled environment (greenhouse)
pop_A = np.random.normal(25, 2, 100)  # Low variation

# Population B: Wild environment
pop_B = np.random.normal(25, 7, 100)  # High variation

# Calculate all measures for both
def calc_dispersion(data, name):
    mean = np.mean(data)
    range_val = np.max(data) - np.min(data)
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    QD = IQR / 2
    mean_dev = np.mean(np.abs(data - mean))
    variance = np.var(data)
    std_dev = np.std(data)
    
    print(f"\n{name}:")
    print(f"   Mean:              {mean:.2f} cm")
    print(f"   Range:             {range_val:.2f} cm")
    print(f"   IQR:               {IQR:.2f} cm")
    print(f"   Quartile Dev:      {QD:.2f} cm")
    print(f"   Mean Deviation:    {mean_dev:.2f} cm")
    print(f"   Variance (σ²):     {variance:.2f} cm²")
    print(f"   Std Deviation (σ): {std_dev:.2f} cm")
    
    return mean, std_dev

print("🌱 Comparing Two Plant Populations:\n")
print("="*50)
mean_A, sd_A = calc_dispersion(pop_A, "🏠 Population A (Greenhouse - Controlled)")
mean_B, sd_B = calc_dispersion(pop_B, "🌲 Population B (Wild - Variable conditions)")
print("="*50)

print(f"\n💡 Interpretation:")
print(f"   Both populations have SAME MEAN ({mean_A:.1f}cm vs {mean_B:.1f}cm)")
print(f"   But VERY DIFFERENT variation:")
print(f"   • Greenhouse (σ={sd_A:.1f}): Consistent growth, controlled conditions")
print(f"   • Wild (σ={sd_B:.1f}): Variable growth, environmental stress")

# Visualize comparison
fig = go.Figure()

fig.add_trace(go.Box(
    y=pop_A,
    name='Greenhouse',
    marker_color='lightgreen',
    boxmean='sd'
))

fig.add_trace(go.Box(
    y=pop_B,
    name='Wild',
    marker_color='lightcoral',
    boxmean='sd'
))

fig.update_layout(
    title="🌱 Comparing Greenhouse vs Wild Populations<br><sub>Same mean, different dispersion</sub>",
    yaxis_title="Plant Height (cm)",
    height=500,
    template='plotly_white'
)

fig.show()

---

## 📋 Part 9: Quick Reference Summary

### When to Use Which Measure?

| Measure | Best For | Strengths | Weaknesses |
|---------|----------|-----------|------------|
| **Range** | Quick rough estimate | Simple, easy | Sensitive to outliers, ignores middle data |
| **IQR/QD** | Skewed data, outliers present | Robust to outliers | Doesn't use all data |
| **Mean Deviation** | Teaching purposes | Intuitive | Not used much in practice |
| **Variance** | Mathematical calculations | Uses all data | Units are squared (hard to interpret) |
| **Std Deviation** | Most situations | Uses all data, same units as data | Sensitive to outliers |

### Coefficient of Variation (CV)

For comparing variability across different scales:

**CV = (σ / μ) × 100%**

In [ ]:
# Example: Comparing variability in mice vs elephants
mice_weight = np.array([20, 22, 24, 26, 28])  # grams
elephant_weight = np.array([4000, 4400, 4800, 5200, 5600])  # kg

# Calculate CV
cv_mice = (np.std(mice_weight) / np.mean(mice_weight)) * 100
cv_elephant = (np.std(elephant_weight) / np.mean(elephant_weight)) * 100

print("🐭 Mice weights (g):", mice_weight)
print(f"   Mean: {np.mean(mice_weight):.1f}g, SD: {np.std(mice_weight):.2f}g")
print(f"   CV: {cv_mice:.1f}%\n")

print("🐘 Elephant weights (kg):", elephant_weight)
print(f"   Mean: {np.mean(elephant_weight):.1f}kg, SD: {np.std(elephant_weight):.2f}kg")
print(f"   CV: {cv_elephant:.1f}%\n")

print("💡 CV allows fair comparison:")
if cv_mice > cv_elephant:
    print(f"   Mice weights are MORE variable relative to their size ({cv_mice:.1f}% vs {cv_elephant:.1f}%)")
else:
    print(f"   Elephant weights are MORE variable relative to their size")

---

## 🎓 Summary

### Key Takeaways:

✅ **Dispersion = Spread** of data around the center  
✅ **Five main measures**: Range, IQR, Mean Dev, Variance, Std Dev  
✅ **Standard Deviation (σ)** is the most commonly used  
✅ **68-95-99.7 Rule** applies to Normal distributions  
✅ **Same mean ≠ Same data!** Must consider spread  
✅ **Use CV** to compare variability across different scales  

### The Pattern Hunter Way:

```
1. Plot your data (see the shape)
2. Calculate center (mean or median)
3. Calculate spread (σ or IQR)
4. Interpret together: μ ± σ tells the story!
```

### Next Notebook:

**04_sampling_techniques.ipynb** - How to collect representative data

---

<div align="center">

**Made with 💚 by The Pattern Hunter Team**

[🏠 Repository](https://github.com/The-Pattern-Hunter/interactive-ecology-biometry) | 
[📓 Previous: Central Tendency](02_central_tendency_analysis.ipynb) | 
[📓 Next: Sampling](04_sampling_techniques.ipynb)

</div>